In [368]:
import requests, bs4, pandas, re, datetime, numpy
from dateutil.parser import parse

In [ ]:
import requests,bs4

r = requests.get('https://southboroughcc.play-cricket.com/website/results/4462297')
r.raise_for_status()
soup = bs4.BeautifulSoup(r.content, 'html.parser')
r.text
rows = []
inningsno = r.text[r.text.find("innings")+len("innings"):r.text.find("-",r.text.find("innings"))]
for i in range(1,12):
    rows.append(soup.select('#innings'+ inningsno +' > div.table-responsive-sm > table.table.standm.table-hover > tbody > tr:nth-child(' + str(i) + ') > td:nth-child(4) > strong'))
## Extracts all batting scores within <strong>

In [ ]:
rows

In [ ]:
str(r.text).find("innings")
str(r.text).find("-",24020)
str(r.text)[24034:24038]
r.text.find("innings")
r.text.find("-",r.text.find("innings"))
r.text[r.text.find("innings")+len("innings"):r.text.find("-",r.text.find("innings"))]
## Finds the innnings number as defined in Play-Cricket

In [ ]:
import requests,bs4

r2 = requests.get('https://southboroughcc.play-cricket.com/website/results/4462297')
r2.raise_for_status()
soup = bs4.BeautifulSoup(r2.content, 'html.parser')
elems = soup.select('#innings5411478 > div.table-responsive-sm > table.table.standm.table-hover > tbody > tr:nth-child(1)')
## batting scorecard table for first innings - nth child 1-11 are each batsman's row

elems[0].find_all('td')[0].find_all('div')[0].find_all('sup') ## find_all('td') finds all of the 'cells' of data in row. 

In [ ]:
import requests,bs4

r3 = requests.get('https://southboroughcc.play-cricket.com/website/results/4462297')
r3.raise_for_status()
soup3 = bs4.BeautifulSoup(r3.content, 'html.parser')
#elems3 = soup3.select('#innings5411478 > table > tbody') ##bowling scorecard for first innings
elems3 = soup3.select('#innings5411478 > table')
#elems3[0].find_all('td')



In [ ]:
import pandas
table = pandas.read_html(str(elems3[0]))[0] #Generates Panda table of bowling scorecard. Much easier to use than html extraction
table[["BOWLER","OVERSO"]]
len(table)
#for i in range(len(table)):
    #print(table.iloc[[i]])
#table.columns.values[1]
table.iloc[[3]].values[0]#[1]



In [699]:
### Overall match stats

##Extracts match point data [Home game, home penalty, home batting, home bowling, home total, away game, away penalty, away batting , away bowling, away total]

GamePoints_data = []

for i in range(1,3):
    GamePoints = soup4.select('#multiCollapseExample' + str(i) + ' > table')
    
    if len(GamePoints) == 0:
        GamePoints_data = ["NaN"]*10
        
    else:
        GamePoints_table = pandas.read_html(str(GamePoints[0]))[0]
        for j in range(len(GamePoints_table)):
            GamePoints_data.append(GamePoints_table.iloc[[j]].values[0][1])

print(GamePoints_data)        
        
##Extracts division and league data [league , division]

DivisionInfo = soup4.select('body > div.breadcrumb-league-wrapper > div.container.breadcrumb-league > div > div.col-sm-12.col-md-6.col-lg-6.text-center.text-lg-left.leaguedetail-left')
DivisionInfo_data = [""]*2

if 'friendly' in str(DivisionInfo[0]).lower():
    DivisionInfo_data = ["Friendly", "Friendly"]

elif DivisionInfo[0].find_all('span')[0].contents[0].strip() == "":
    DivisionInfo_data = [DivisionInfo[0].find_all('span')[2].contents[0].strip()]*2
    
elif 'https' in str(DivisionInfo[0].find_all('span')[2].contents[1]).lower():
    DivisionInfo_data[0] = DivisionInfo[0].find_all('span')[0].contents[0].strip()
    start = str(DivisionInfo[0].find_all('span')[2].contents[1]).find(">") + 1 
    end = str(DivisionInfo[0].find_all('span')[2].contents[1]).find("<",str(DivisionInfo[0].find_all('span')[2].contents[1]).find(">")) 
    DivisionInfo_data[1] = str(DivisionInfo[0].find_all('span')[2].contents[1])[start:end]
    
print(DivisionInfo_data)

##Extracts date and location data [ground , date]

DateGround_data = [""]*2

DateGround = soup4.select('body > div.breadcrumb-league-wrapper > div.container.breadcrumb-league > div > div.col-sm-12.col-md-6.col-lg-6.text-lg-right.leaguedetail-right')
Date = parse(re.search('\d*? \w* \d{4}',DateGround[0].text)[0])
Ground = DateGround[0].find_all('a')[0].contents[0]

if "\n" not in Ground:
    DateGround_data[0] = Ground
DateGround_data[1] = Date.date()

print(DateGround_data)



## Extracts [Home team, Away team, Home batting score, Away batting score, Home wickets lost, Away wickets lost, Home overs, Away overs, Winner, by runs, by wickets, other, toss winner, toss decision]

Teams = soup4.select('body > div.container.main-header.main-header-lg.d-none.d-lg-block > table')

Teams_data = [""]*14

for i in range(2):
    Clubname = Teams[0].find_all('p', class_="team-name")[i].contents[0]
    first = str(Teams[0].find_all('p', class_="team-info-2")[i].find_all(class_ = "team-info-1")[0]).find("\n")
    second = str(Teams[0].find_all('p', class_="team-info-2")[i].find_all(class_ = "team-info-1")[0]).find("\n", first + 1)
    Clubteam = str(Teams[0].find_all('p', class_="team-info-2")[i].find_all(class_ = "team-info-1")[0])[first+1:second].strip()
    Teams_data[i] = Clubname + ' ' + Clubteam


for i in range(2):
    first = str(Teams[0].find_all('p', class_ = "team-info-2")[i]).find("</span>\n")
    second = str(Teams[0].find_all('p', class_ = "team-info-2")[i]).find("<", first + 1)
    Battingscore = str(Teams[0].find_all('p', class_ = "team-info-2")[i])[first + len("</span>\n") : second].strip()
    Teams_data[2 + i] = Battingscore
    

for i in range(2):
    if len(Teams[0].find_all('p', class_ = "team-info-2")[i].find_all(class_ = "smalltxt")) == 0:
        Teams_data[4 + i], Teams_data[6 + i] = "",""
    else :
        WicketsOvers_html = Teams[0].find_all('p', class_ = "team-info-2")[i].find_all(class_ = "smalltxt")[0].contents[0]
        first = WicketsOvers_html.find("/")
        second = WicketsOvers_html.find("(", first + 1)
        third = WicketsOvers_html.find(")", second + 1)
        Wickets = WicketsOvers_html[first + 1 : second].strip()
        if "All out" in Wickets:
            Teams_data[4 + i] = 10
        else :
            Teams_data[4 + i] = Wickets
        Overs = WicketsOvers_html[second + 1 : third].strip()
        Teams_data[6 + i] = Overs

        
if "ABANDONED" in str(Teams[0]):
    Teams_data[8] = "None"
    Teams_data[11] = "Abandoned"

elif "CONCEDED" in str(Teams[0]):
    Teamconceded = Teams[0].find_all('p', class_ = "match-ttl win-cb-name")[0].contents[0].strip()
    if Teamconceded in Teams_data[0].upper():
        Teams_data[8] = Teams_data[1]
    else :
        Teams_data[8] = Teams_data[0]
    Teams_data[11] = "Conceded"
    
else :
    Matchwinner = Teams[0].find_all('p', class_ = "match-ttl win-cb-name")[0].contents[0].strip()
    first = str(Teams[0].find_all('div', class_ = "info mdont")[0].contents[1]).find("<span>")
    second = str(Teams[0].find_all('div', class_ = "info mdont")[0].contents[1]).find("<", first + 1)
    Wintype = str(Teams[0].find_all('div', class_ = "info mdont")[0].contents[1])[first + len("<span>") : second]
    Byhowmuch = ''.join(filter(str.isdigit, str(Teams[0].find_all('div', class_ = "info mdont")[0].contents[0])))

    if Matchwinner in Teams_data[0].upper():
        Teams_data[8] = Teams_data[0]
    else :
        Teams_data[8] = Teams_data[1]
    if Wintype == "RUNS":
            Teams_data[9] = Byhowmuch
    if Wintype == "WICKETS":
            Teams_data[10] = Byhowmuch      

            
if len(Teams[0].find_all('p', class_ = "team-info-3")) != 0:
    for i in range(2):
        if len(Teams[0].find_all('p', class_ = "team-info-3")[i].contents) != 0:
            Teams_data[12] = Teams_data[i]
            if "bat" in Teams[0].find_all('p', class_ = "team-info-3")[i].contents[0]:
                Teams_data[13] = "Bat"
            else :
                Teams_data[13] = "Field"

    if Teams_data[12] == "" and Teams_data[13] == "":
        Teams_data[12], Teams_data[13] = "No toss", "No toss"
        
elif len(Teams[0].find_all('p', class_ = "team-info-3 adma")) != 0:
    for i in range(2):
        if len(Teams[0].find_all('p', class_ = "team-info-3 adma")[i].contents) != 0:
            Teams_data[12] = Teams_data[i]
            if "bat" in Teams[0].find_all('p', class_ = "team-info-3 adma")[i].contents[0]:
                Teams_data[13] = "Bat"
            else :
                Teams_data[13] = "Field"

    if Teams_data[12] == "" and Teams_data[13] == "":
        Teams_data[12], Teams_data[13] = "No toss", "No toss"            
            
print(Teams_data)

[0, 0, 3, 5, 8, 20, 0, 0, 0, 20]
['KENT COUNTY VILLAGE LEAGUE', 'Corona League Tier 2 Division A']
['Limpsfield Chart Cricket Club', datetime.date(2020, 8, 22)]
['Limpsfield Chart CC 1st XI', 'Southborough CC 1st XI', '184', '209', 10, 10, '39.4', '40', 'Southborough CC 1st XI', '25', '', '', 'Limpsfield Chart CC 1st XI', 'Field']


In [698]:
### Test webpages ####


#r4 = requests.get('https://southboroughcc.play-cricket.com/website/results/4462297') # Division / League
#r4 = requests.get('https://southboroughcc.play-cricket.com/website/results/4428817') # Friendly
#r4 = requests.get('https://southboroughcc.play-cricket.com/website/results/27378') # No division just league
#r4 = requests.get('https://southboroughcc.play-cricket.com/website/results/412586') # no division, no ground, abandoned
#r4 = requests.get('https://southboroughcc.play-cricket.com/website/results/4413968') # u13, conceded
#r4 = requests.get('https://southboroughcc.play-cricket.com/website/results/4462178') # team won tost and field first, and won by runs

r4.raise_for_status()
soup4 = bs4.BeautifulSoup(r4.content, 'html.parser')

['Limpsfield Chart CC 1st XI', 'Southborough CC 1st XI', '184', '209', 10, 10, '39.4', '40', 'Southborough CC 1st XI', '25', '', '', 'Limpsfield Chart CC 1st XI', 'Field']


In [675]:
len(Teams[0].find_all('p', class_ = "team-info-3 adma")[0].contents)

1